In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta

# Data Prep


In [2]:
#load data
df_ntc = pd.read_csv("../data/NTC.csv")
# display(df_ntc)

df_act_loss = pd.read_csv("../data/Avtice-losses.csv")
df_act_loss = df_act_loss.rename(columns={"Wirkverluste/Active Losses": "loss", "Unnamed: 0": "datetime"})
df_act_loss = df_act_loss[1:].reset_index()
df_act_loss = df_act_loss.drop(['index'], axis=1)
# display(df_act_loss)

df_weather_forecast = pd.read_csv("../data/Forecast-renewable-generation.csv")

df_weather_temp = pd.read_csv("../data/Forecast-temperature.csv")
# display(df_weather_temp)

In [3]:
# df_ntc
#set index
df_ntc['datetime'] = pd.to_datetime(df_ntc['datetime'])
df_ntc.set_index('datetime', inplace=True)

# Drop duplicate datetimes
# duplicates = df_ntc.index[df_ntc.index.duplicated()].unique()
# df_ntc = df_ntc.loc[~df_ntc.index.duplicated(keep='first')]

In [4]:
# df_act_loss
#set index
df_act_loss['datetime'] = pd.to_datetime(df_act_loss['datetime'])
df_act_loss['datetime'] = df_act_loss['datetime'] - pd.Timedelta(minutes=15)
df_act_loss.set_index('datetime', inplace=True)

# You need to convert kWh to MWh by dividing it by 1000. As all other variables use MW.
df_act_loss['loss'] = df_act_loss['loss'].astype(float)
df_act_loss['loss'] = df_act_loss['loss'] / 1000.0

#dowsample from 1 hour to 15 min
df_act_loss = df_act_loss.loc[~df_act_loss.index.duplicated(keep='first')]
df_act_loss = df_act_loss.resample('1H').sum()
# Interpolate to fill in the missing values
df_act_loss = df_act_loss.interpolate()

In [5]:
# df_weather_temp
#set index
df_weather_temp['datetime'] = pd.to_datetime(df_weather_temp['datetime'])
df_weather_temp.set_index('datetime', inplace=True)

#drop duplicates
duplicates = df_weather_temp.index[df_weather_temp.index.duplicated()].unique()
print(duplicates)
df_weather_temp = df_weather_temp.loc[~df_weather_temp.index.duplicated(keep='first')]

# Upsample to 1-hour frequency with NaNs
df_weather_temp = df_weather_temp.resample('1H').asfreq()

# Cubic interpolation to fill NaNs smoothly
df_weather_temp = df_weather_temp.interpolate(method='cubic')
df_weather_temp

DatetimeIndex(['2021-10-31 02:00:00'], dtype='datetime64[ns]', name='datetime', freq=None)


,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
datetime,,,,
2019-01-01 01:00:00,4.106700,5.972900,7.426800,4.028100
2019-01-01 02:00:00,2.637812,5.080813,6.724312,2.372176
2019-01-01 03:00:00,1.701501,4.600184,6.284190,1.437560
2019-01-01 04:00:00,1.221641,4.468391,6.066192,1.115047
2019-01-01 05:00:00,1.122105,4.622812,6.030072,1.295434
...,...,...,...,...
2021-12-31 19:00:00,8.530000,9.960000,10.790000,9.590000
2021-12-31 20:00:00,8.000000,9.400000,10.630000,9.110000
2021-12-31 21:00:00,7.500000,8.880000,10.510000,8.670000


In [6]:
# df_weather_forecast
#set index
df_weather_forecast['datetime'] = pd.to_datetime(df_weather_forecast['datetime'])
df_weather_forecast.set_index('datetime', inplace=True)

# MODEL

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

In [8]:
#load data
df_ntc
# df_act_loss

# df_weather_forecast

# df_weather_temp
display(df_ntc)
display(df_weather_temp)
display(df_weather_forecast)
display(df_act_loss)

# List of DataFrames
dfs = [df_ntc, df_weather_temp, df_weather_forecast, df_act_loss]

# Merge all DataFrames on 'datetime'
from functools import reduce
merged_df = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='inner'), dfs)

,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH
datetime,,,,,,,,
2019-01-01 00:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
2019-01-01 01:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
2019-01-01 02:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
2019-01-01 03:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
2019-01-01 04:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0
...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0
2021-12-31 20:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0
2021-12-31 21:00:00,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0


,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
datetime,,,,
2019-01-01 01:00:00,4.106700,5.972900,7.426800,4.028100
2019-01-01 02:00:00,2.637812,5.080813,6.724312,2.372176
2019-01-01 03:00:00,1.701501,4.600184,6.284190,1.437560
2019-01-01 04:00:00,1.221641,4.468391,6.066192,1.115047
2019-01-01 05:00:00,1.122105,4.622812,6.030072,1.295434
...,...,...,...,...
2021-12-31 19:00:00,8.530000,9.960000,10.790000,9.590000
2021-12-31 20:00:00,8.000000,9.400000,10.630000,9.110000
2021-12-31 21:00:00,7.500000,8.880000,10.510000,8.670000


,solar_fore_de [MW],solar_fore_it [MW],wind_fore_de [MW],wind_fore_it [MW]
datetime,,,,
2019-01-01 00:00:00,0.0,0.0,21344.8514,4302.6977
2019-01-01 01:00:00,0.0,0.0,23052.3310,4596.5916
2019-01-01 02:00:00,0.0,0.0,24969.9701,4478.5564
2019-01-01 03:00:00,0.0,0.0,27082.9626,4323.3712
2019-01-01 04:00:00,0.0,0.0,26890.9717,4231.8283
...,...,...,...,...
2021-12-31 19:00:00,0.0,0.0,36997.7200,1108.4000
2021-12-31 20:00:00,0.0,0.0,35666.9300,1077.9700
2021-12-31 21:00:00,0.0,0.0,34383.8800,1048.2800


,loss
datetime,
2019-01-01 00:00:00,139.525004
2019-01-01 01:00:00,129.716036
2019-01-01 02:00:00,133.398074
2019-01-01 03:00:00,135.133852
2019-01-01 04:00:00,131.699424
...,...
2021-12-31 19:00:00,171.707318
2021-12-31 20:00:00,159.462903
2021-12-31 21:00:00,155.109520


In [9]:
#remove duplicate + renamed
duplicates = merged_df.index[merged_df.index.duplicated()].unique()
print(duplicates)
merged_df = merged_df.loc[~merged_df.index.duplicated(keep='first')]

merged_df.columns = merged_df.columns.str.replace('[":\[\]{}]', '', regex=True)

merged_df

DatetimeIndex(['2019-10-27 02:00:00', '2020-10-25 02:00:00',
               '2021-10-31 02:00:00'],
              dtype='datetime64[ns]', name='datetime', freq=None)


,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,solar_fore_de MW,solar_fore_it MW,wind_fore_de MW,wind_fore_it MW,loss
datetime,,,,,,,,,,,,,,,,,
2019-01-01 01:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,4.106700,5.972900,7.426800,4.028100,0.0,0.0,23052.3310,4596.5916,129.716036
2019-01-01 02:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,2.637812,5.080813,6.724312,2.372176,0.0,0.0,24969.9701,4478.5564,133.398074
2019-01-01 03:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,1.701501,4.600184,6.284190,1.437560,0.0,0.0,27082.9626,4323.3712,135.133852
2019-01-01 04:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,1.221641,4.468391,6.066192,1.115047,0.0,0.0,26890.9717,4231.8283,131.699424
2019-01-01 05:00:00,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,1.122105,4.622812,6.030072,1.295434,0.0,0.0,27740.1555,4266.3082,147.391128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,8.530000,9.960000,10.790000,9.590000,0.0,0.0,36997.7200,1108.4000,171.707318
2021-12-31 20:00:00,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,8.000000,9.400000,10.630000,9.110000,0.0,0.0,35666.9300,1077.9700,159.462903
2021-12-31 21:00:00,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,7.500000,8.880000,10.510000,8.670000,0.0,0.0,34383.8800,1048.2800,155.109520


In [10]:
merged_df.columns

Index(['CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
       'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
       'temperature_fore_it', 'solar_fore_de MW', 'solar_fore_it MW',
       'wind_fore_de MW', 'wind_fore_it MW', 'loss'],
      dtype='object')

In [11]:
def determine_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

# Create a 'season' column in the DataFrame
season_series = pd.Series(merged_df.index.month).apply(determine_season)

# Initialize columns for individual seasons
merged_df['Winter'] = 0
merged_df['Spring'] = 0
merged_df['Summer'] = 0
merged_df['Autumn'] = 0

# Set the corresponding season column to 1
for idx, season in enumerate(season_series):
    merged_df.at[merged_df.index[idx], season] = 1

In [12]:
df = merged_df


# # Create lagged features based on the past 7 days (7 * 24 hours)
# Define the columns you want to create lags and moving averages for
feature_columns = [
    'loss',
    'temperature_fore_ch',
    'temperature_fore_fr',
    'temperature_fore_de',
    'temperature_fore_it',
    'solar_fore_de MW',
    'solar_fore_it MW',
    'wind_fore_de MW',
    'wind_fore_it MW',
]

# Create lagged features
for col in feature_columns:
    for i in range(1, 7 * 24 + 1):  # for past 7 days (in hours)
        df[f'lag_{col}_{i}'] = df[col].shift(i)

# Create moving averages for each feature
for col in feature_columns:
    df[f'ma_{col}_6hr'] = df[col].rolling(window=6).mean()  # 6-hour moving average
    df[f'ma_{col}_12hr'] = df[col].rolling(window=12).mean()  # 12-hour moving average
    df[f'ma_{col}_1day'] = df[col].rolling(window=24).mean()  # 1-day moving average
    df[f'ma_{col}_3day'] = df[col].rolling(window=72).mean()  # 3-day moving average
    df[f'ma_{col}_7days'] = df[col].rolling(window=24*7).mean()  # 7-day moving average

# Drop rows with NaN values introduced by lagging and moving averages

df['target'] = df['loss'].shift(-24)
# Drop NaN values introduced by lagging
df.dropna(inplace=True)
df

/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, u

/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, u

/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, u

/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_35182/821433269.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, u

,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,temperature_fore_ch,temperature_fore_fr,...,ma_wind_fore_de MW_12hr,ma_wind_fore_de MW_1day,ma_wind_fore_de MW_3day,ma_wind_fore_de MW_7days,ma_wind_fore_it MW_6hr,ma_wind_fore_it MW_12hr,ma_wind_fore_it MW_1day,ma_wind_fore_it MW_3day,ma_wind_fore_it MW_7days,target
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-08 01:00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,2.593200,6.616400,...,26653.476200,17977.425713,15891.437628,19551.899959,2242.366683,2605.069867,2401.358313,2488.230689,3368.933288,172.738969
2019-01-08 02:00:00,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,2.531251,6.660703,...,28375.422858,19037.361442,16004.101533,19605.382916,2031.048750,2467.221942,2379.475512,2475.547069,3351.545437,192.147994
2019-01-08 03:00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,2.388536,6.691293,...,29984.488050,20108.561162,16115.612294,19651.570874,1805.742700,2301.785058,2348.747183,2459.175506,3332.517738,188.856706
2019-01-08 04:00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,2.224649,6.736970,...,31496.057217,21177.895425,16208.965747,19704.131012,1586.399150,2131.154458,2312.050825,2440.139264,3312.905030,183.046813
2019-01-08 05:00:00,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,2.099185,6.826530,...,32849.026825,22319.165346,16320.495431,19757.832736,1420.626867,1982.996292,2274.121283,2420.168806,3293.485380,199.166533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-30 19:00:00,1200.0,4000.0,1400.0,4320.0,1200.0,800.0,3200.0,1810.0,10.010000,11.460000,...,34605.747500,30267.731250,23402.361667,19805.886964,4194.746667,3711.892500,3169.990000,2973.960972,3214.892143,171.707318
2021-12-30 20:00:00,1200.0,4000.0,1400.0,4320.0,1200.0,800.0,3200.0,1810.0,9.550000,10.780000,...,35201.105000,30970.822917,23578.928194,19853.237202,4284.733333,3870.552500,3230.596250,2996.355694,3232.217619,159.462903
2021-12-30 21:00:00,1200.0,4000.0,1400.0,4094.0,1200.0,800.0,3200.0,1810.0,9.170000,10.280000,...,35719.638333,31597.289583,23737.690000,19891.395119,4332.700000,4023.698333,3289.803333,3014.408056,3248.300952,155.109520


In [ ]:
# Sort the DataFrame by datetime
df.sort_index(inplace=True)

# Train-test split based on timestamp
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Features and target
X_train, y_train = train.drop('target', axis=1), train['target']
X_test, y_test = test.drop('target', axis=1), test['target']

# LightGBM parameters
params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'device': 'gpu',
    'random_state': 42
}

# Create LightGBM data containers
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Train the model
bst = lgb.train(params, train_data, valid_sets=[test_data])

# Make predictions
y_pred = bst.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 399967
[LightGBM] [Info] Number of data points in the train set: 20886, number of used features: 1578
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1574 dense feature groups (31.39 MB) transferred to GPU in 0.158253 secs. 1 sparse feature groups
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 106.915617


In [ ]:
y_pred_df = pd.DataFrame({'pred_loss': y_pred}, index=X_test.index)

# 2. Reshape the DataFrame
# Group by date and create a dictionary
pred_dict = {}
for date, group in y_pred_df.groupby(y_pred_df.index.date):
    daily_preds = group['pred_loss'].values
    if len(daily_preds) == 24:  # Only take days that have full 24 hours data
        pred_dict[date] = daily_preds

# 3. Save each day's observations in a dictionary as specified
output_dict = {}
for date, hourly_preds in pred_dict.items():
    base_datetime = datetime.combine(date, datetime.min.time())
    output_dict[str(date)] = {
        (base_datetime + timedelta(hours=hour)).strftime("%Y-%m-%d %H:%M:%S"): pred
        for hour, pred in enumerate(hourly_preds)
    }

# Print the result (or you can save it to a file)
print(output_dict)

In [ ]:
output_dict

In [ ]:
y_pred